In [1]:
#!/usr/bin/env python3
"""
Neven Caplar 
Last updated: 2023-12-01

Goals: 
Fit the data

Each Section can/should run independently,
only these initial imports should be shared among all sections

Open questions:
None at the moment
"""

import os

import numpy as np
import pandas as pd
import pyarrow as pa

# from scipy.spatial import KDTree
import matplotlib.pyplot as plt

import JaxPeriodDrwFit


from tape.ensemble import Ensemble
from tape.utils import ColumnMapper

from tqdm import tqdm

from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [2]:
import dask
# many workers
# dask.config.set(scheduler='threads') 

dask.config.set({"temporary-directory" :'/epyc/ssd/users/ncaplar/tmp'})

# does not work
# from multiprocessing.pool import ThreadPool
# dask.config.set(pool=ThreadPool(20))

# one worker
# dask.config.set(scheduler='processes')  
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=12, threads_per_worker=1)
client = Client(cluster)
# cluster.adapt(minimum=10, maximum=40) 

/astro/users/ncaplar/.conda/envs/tape_static/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40712 instead
  warnings.warn(


In [3]:
def pack_output_to_parquet(result, cols, output_dir, output_filename, drop_cols=[], full=False):
    """Packs output to a dataframe, written as a parquet file. The created dataframe object is returned for inspection."""
    result_df = None
    if full:
        # Construct dataframes with the results for each object.
        dfs = []
        for i in range(len(result)):
            obj_data = result.iloc[i]
            # Construct a series representing the index
            obj_index = pd.Series(np.full(len(obj_data), result.index[i]), name=result.index.name)
            dfs.append(pd.DataFrame(data=obj_data, columns=cols, index=obj_index))

        # Concatenate all of the per-object dataframes
        result_df = pd.concat(dfs)
    else:
        # Each object only has a 1D array in the result series, so the constructed
        # dataframe has the same number of rows. So we can just do a 1:1 mapping with column names 
        result_df = pd.DataFrame(columns=cols, index=result.index)
        for i in range(len(result)):
            result_df.iloc[i] = result[i]

    # Drop any columns if requested.
    if drop_cols:
        result_df = result_df.drop(columns=drop_cols)

    # Write the output to the parquet file
    pa_table = pa.Table.from_pandas(result_df)
    pa_table
    pa.parquet.write_table(pa_table, f"{output_dir}/data/{output_filename}.parquet")
    return result_df

# Create columns for result of using just the drw kernel
param_cols = ['log_drw_scale', 'log_drw_amp']
init_param_cols = ["init_" + c for c in param_cols]
drw_columns = ['min_neg_log_lh', 'neg_log_lh'] + param_cols + init_param_cols

# Create columns for result of combining the drw params with periodic params
param_cols = ['log_drw_scale', 'log_drw_amp', 'log_per_scale', 'log_per_amp']
init_param_cols = ["init_" + c for c in param_cols]
combined_columns = ['min_neg_log_lh', 'neg_log_lh'] + param_cols + init_param_cols

In [8]:
ens = Ensemble(client = client)  # initialize an ensemble object
ens.client_info()

ens_drw = Ensemble(client = client)  # initialize an ensemble object
ens_drw.client_info()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:43967/status,
Dashboard: http://127.0.0.1:43967/status,Workers: 12
Total threads: 12,Total memory: 251.68 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41585,Workers: 12
Dashboard: http://127.0.0.1:43967/status,Total threads: 12
Started: 4 minutes ago,Total memory: 251.68 GiB
Comm: tcp://127.0.0.1:46530,Total threads: 1
Dashboard: http://127.0.0.1:40261/status,Memory: 20.97 GiB
Nanny: tcp://127.0.0.1:40271,


In [5]:
dask.config.get("temporary-directory")

'/epyc/ssd/users/ncaplar/tmp'

In [6]:
# Setup base directory for saving output files
username= "ncaplar"
basedir = f"/astro/users/{username}/data/"

# Simulations

In [10]:
# if running on baldur or epyc
data_path = "/epyc/users/ncaplar/data/drw_per_sim/"

col_map = ColumnMapper(id_col="lc_nr", 
                       time_col="t",
                       flux_col="y", 
                       err_col="yerr",
                       band_col="band")

ens.from_parquet(data_path,
                 source_subdir="sources",
                 column_mapper=col_map,
                 additional_cols=True,
                 sync_tables=True,
                 sorted=True
                 )


col_map = ColumnMapper(id_col="lc_nr", 
                       time_col="t",
                       flux_col="y_drw", 
                       err_col="yerr",
                       band_col="band")
# do I actually need to do this?
ens_drw.from_parquet(data_path,
                 source_subdir="sources",
                 column_mapper=col_map,
                 additional_cols=True,
                 sync_tables=True,
                 sorted=True
                 )

In [11]:
ens_source = ens._source.compute()

In [12]:
ens_source

,drw_amp,drw_tau,per_amp,per_tau,t,y,yerr,y_drw,y_per,y_drw_like_y,yerr_drw,band
lc_nr,,,,,,,,,,,,
0,0.116043,562.082417,0.000000,9.999999e+06,0.000000,15.235764,0.015961,15.215507,0.000000,15.217444,0.015961,g
0,0.116043,562.082417,0.000000,9.999999e+06,7.056726,15.228592,0.016030,15.205763,0.000000,15.216823,0.016030,g
0,0.116043,562.082417,0.000000,9.999999e+06,9.997028,15.221954,0.016041,15.204183,0.000000,15.220355,0.016041,g
0,0.116043,562.082417,0.000000,9.999999e+06,14.113451,15.194743,0.016057,15.201953,0.000000,15.198695,0.016057,g
0,0.116043,562.082417,0.000000,9.999999e+06,17.053754,15.215950,0.015966,15.214809,0.000000,15.215787,0.015966,g
...,...,...,...,...,...,...,...,...,...,...,...,...
99,0.116043,562.082417,0.091111,5.760088e+01,1504.062670,15.308652,0.015494,15.201424,0.083237,15.208314,0.016061,g
99,0.116043,562.082417,0.091111,5.760088e+01,1506.022871,15.281972,0.015537,15.204586,0.073480,15.197522,0.016039,g
99,0.116043,562.082417,0.091111,5.760088e+01,1508.963174,15.217295,0.015834,15.180938,0.052749,15.184592,0.016210,g


In [16]:
JaxPeriodDrwFit_instance = JaxPeriodDrwFit.JaxPeriodDrwFit()

# Result simulation Fullmodel Fulldata 
res_sim_full_full = ens.batch(JaxPeriodDrwFit_instance.optimize_map, 't', "y", "yerr",
                compute=True, meta=None, n_init=100, full=True)

# Result simulation Fullmodel DRWdata
res_sim_full_drw = ens.batch(JaxPeriodDrwFit_instance.optimize_map, 't', "y_drw_like_y", "yerr_drw",
                compute=True, meta=None, n_init=100, full=True)

# hm, why do I need to reinstantiate this?
JaxPeriodDrwFit_instance = JaxPeriodDrwFit.JaxPeriodDrwFit()

# Result simulation DRWmodel Fulldata
res_sim_drw_full = ens.batch(JaxPeriodDrwFit_instance.optimize_map_drw, 't', "y", "yerr",
                compute=True, meta=None, n_init=100, full=True)

# Result simulation DRWmodel DRWdata
res_sim_drw_drw = ens.batch(JaxPeriodDrwFit_instance.optimize_map_drw, 't', "y_drw_like_y", "yerr_drw",
                compute=True, meta=None, n_init=100, full=True)

In [18]:
# Save outputs
df_drw_full = pack_output_to_parquet(res_sim_drw_full, drw_columns,
                       f"/astro/users/{username}", "res_sim_run_g_0_drw_full", full=True)

# Save output for results from the combined drw and periodic kernel
df_full_full = pack_output_to_parquet(res_sim_full_full, combined_columns,
                       f"/astro/users/{username}", "res_sim_run_g_0_full_full", full=True)

df_full_drw = pack_output_to_parquet(res_sim_full_drw, combined_columns,
                       f"/astro/users/{username}", "res_sim_run_g_0_full_drw", full=True)

df_drw_drw = pack_output_to_parquet(res_sim_drw_drw, drw_columns,
                       f"/astro/users/{username}", "res_sim_run_g_0_drw_drw,", full=True)

In [51]:
res_tsp.loc[0].iloc[0]

Array([-365.54909544, -365.54909544,    2.46311869,   -1.35135244,
          2.52358148,   -1.64046852,    2.46897798,   -1.25395213,
          2.6112163 ,   -0.44041735], dtype=float64)

In [52]:
res_tsp.loc[0].iloc[1]

Array([-355.78656511, -355.78656511,    2.29947412,   -1.24087935,
          3.19145422,   -1.13794723,    1.94338645,   -1.16108434,
          3.07503613,   -1.9325327 ], dtype=float64)

In [ ]:
drw_df.loc[0].iloc[0]

In [47]:
drw_df_full.loc[0].iloc[0]

min_neg_log_lh       -364.357856
neg_log_lh           -364.357856
log_drw_scale           2.437787
log_drw_amp            -1.281439
init_log_drw_scale      1.872701
init_log_drw_amp       -2.842854
Name: 0, dtype: float64

In [48]:
drw_df_full.loc[0].iloc[100+0]

min_neg_log_lh       -354.307725
neg_log_lh           -342.820118
log_drw_scale           2.052512
log_drw_amp            -1.632540
init_log_drw_scale      1.872701
init_log_drw_amp       -2.842854
Name: 0, dtype: float64

# Test code that will go in the script

In [4]:
ens = Ensemble(client = client)  # initialize an ensemble object
ens.client_info()

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:40712/status,
Dashboard: http://127.0.0.1:40712/status,Workers: 12
Total threads: 12,Total memory: 251.68 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35540,Workers: 12
Dashboard: http://127.0.0.1:40712/status,Total threads: 12
Started: 1 minute ago,Total memory: 251.68 GiB
Comm: tcp://127.0.0.1:32905,Total threads: 1
Dashboard: http://127.0.0.1:40177/status,Memory: 20.97 GiB
Nanny: tcp://127.0.0.1:36360,


In [11]:
from pyarrow import parquet
parquet.read_schema("/epyc/users/ncaplar/data/drw_per_sim/sources/sim_drw_023922.87-000119.6.parquet", memory_map=True)

id: string
lc_nr: int64
drw_amp: double
drw_tau: double
t: double
y: double
yerr: double
band: string
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 1127

In [26]:
# if running on baldur or epyc
data_path = "/epyc/users/ncaplar/data/drw_per_sim/"

col_map = ColumnMapper(id_col="lc_nr", 
                       time_col="t",
                       flux_col="y", 
                       err_col="yerr",
                       band_col="band")

ens.from_parquet(data_path,
                 source_subdir="sources",
                 column_mapper=col_map,
                 additional_cols=True,
                 sync_tables=True,
                 sorted=True
                 )

In [34]:
JaxPeriodDrwFit_instance = JaxPeriodDrwFit.JaxPeriodDrwFit()


# Result simulation Fullmodel DRWdata
res_sim_full_drw = ens.batch(JaxPeriodDrwFit_instance.optimize_map, 't', "y", "yerr",
                compute=True, meta=None, n_init=100, full=False)

Using generated label, result_3, for a batch result.


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [65]:
res_sim_full_drw

lc_nr
0     [-366.5278803044778, -366.5278803044778, 2.267...
1     [-347.60062161095084, -347.60062161095084, 1.6...
2     [-361.9964027378525, -361.9964027378525, 2.912...
3     [-363.825764126508, -363.825764126508, 2.11146...
4     [-354.97570071945836, -354.97570071945836, 3.0...
                            ...                        
95    [-365.18299644808934, -365.18299644808934, 2.2...
96    [-345.69063344485124, -345.69063344485124, 2.8...
97    [-376.78846501201684, -376.78846501201684, 8.1...
98    [-347.68162148983566, -347.68162148983566, 1.9...
99    [-351.53466556329334, -351.53466556329334, 2.0...
Name: lc_nr, Length: 100, dtype: object

In [74]:
data = res_sim_full_drw.to_frame()['lc_nr'].tolist()

# Create a new DataFrame with 10 columns
new_df = pd.DataFrame(data, columns=['min_ln_abs', 'min_ln',
                                     'log_drw_scale', 'log_drw_amp',
                                     'log_per_scale', 'log_per_amp',
                                     'log_drw_scale_init', 'log_drw_amp_init',
                                     'log_per_scale_init', 'log_per_amp_init'])

# Concatenate the new DataFrame with the original DataFrame, dropping the original list column
df = pd.concat([res_sim_full_drw.to_frame(), new_df], axis=1).drop('lc_nr', axis=1)


# List of columns to convert to float
columns_to_convert = ['min_ln_abs', 'min_ln', 'log_drw_scale', 'log_drw_amp', 'log_per_scale', 
                      'log_per_amp', 'log_drw_scale_init', 'log_drw_amp_init', 
                      'log_per_scale_init', 'log_per_amp_init']

# Convert specified columns to float using apply and pd.to_numeric
df[columns_to_convert] = df[columns_to_convert].values.tolist()

# Export DataFrame to Parquet
# df.to_parquet('/epyc/users/ncaplar/data/drw_per_sim/results_fit_sim'+str(023922.87-000119.6), index=False)

df.dtypes

min_ln_abs            object
min_ln                object
log_drw_scale         object
log_drw_amp           object
log_per_scale         object
log_per_amp           object
log_drw_scale_init    object
log_drw_amp_init      object
log_per_scale_init    object
log_per_amp_init      object
dtype: object

In [76]:
df['min_ln_abs']

0      -366.5278803044778
1     -347.60062161095084
2      -361.9964027378525
3       -363.825764126508
4     -354.97570071945836
             ...         
95    -365.18299644808934
96    -345.69063344485124
97    -376.78846501201684
98    -347.68162148983566
99    -351.53466556329334
Name: min_ln_abs, Length: 100, dtype: object

In [59]:
df.to_parquet('/epyc/users/ncaplar/data/drw_per_sim/results_fit_sim'+str(023922.87-000119.6))

AttributeError: 'Series' object has no attribute 'to_parquet'